In [1]:
import os
import emoji
import tweepy

#https://medium.freecodecamp.org/how-to-build-a-twitter-sentiments-analyzer-in-python-using-textblob-948e1e8aae14
from textblob import TextBlob

from IPython.core.display import clear_output
from time import time

import plotly
import bokeh

# import plotly

In [2]:
# topic modelling

import gensim, spacy,nltk
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis.gensim 

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



In [39]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
newsgroup_test = fetch_20newsgroups(subset = 'test')
categories = newsgroup_test.target_names
categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [2]:
# from apikey import TWITTER_KEY,TWITTER_SECRET,TWITTER_TOKEN,TWITTER_TOKEN_SECRET

In [44]:
news = fetch_20newsgroups(shuffle=True,random_state = 1,
                          remove=('headers', 'footers', 'quotes'),
                         categories = categories)
data = news.data
print(len(data))
data[:2]
data_word = list(map(gensim.utils.simple_preprocess,data))
print(len(sum(data_word,[])))

11314
2057329


In [56]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['com', 'from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
print(len(stop_words))



222


process： tokenize -> remove stop -> lemmanize --> remove stop again

In [50]:
data_words = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in data_word]

In [51]:
len(sum(data_words,[]))

1153799

In [61]:
text2

'Catching Criminal Minds and I screamed so loudly my roomie came running in 😂😂 #impathetic #criminalminds'

In [59]:
text2.split(' ')

['Catching',
 'Criminal',
 'Minds',
 'and',
 'I',
 'screamed',
 'so',
 'loudly',
 'my',
 'roomie',
 'came',
 'running',
 'in',
 '😂😂',
 '#impathetic',
 '#criminalminds']

In [65]:
data_sim = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text2.split(' ')]

In [70]:
data_sim

[['catching'],
 ['criminal'],
 ['minds'],
 [],
 [],
 ['screamed'],
 [],
 ['loudly'],
 [],
 ['roomie'],
 ['came'],
 ['running'],
 [],
 [],
 ['impathetic'],
 ['criminalminds']]

In [69]:
data_ready = []
nlp = spacy.load('en', disable=['parser', 'ner'])
allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
for sent in data_sim:
    doc = nlp(' '.join(sent))

    data_ready.append(
        [token.lemma_ for token in doc if token.pos_ in allowed_postags])
data_ready = [[word for word in simple_preprocess(str(doc))
               if word not in stop_words] for doc in data_ready]
data_ready

[['catch'],
 ['criminal'],
 ['mind'],
 [],
 [],
 ['scream'],
 [],
 ['loudly'],
 [],
 ['roomie'],
 [],
 [],
 [],
 [],
 ['impathetic'],
 ['criminalmind']]

In [86]:
# step two feature Extraction 
id2word = corpora.Dictionary(data_ready)
corpus = [id2word.doc2bow(text) for text in data_ready]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,
                                           num_topics=1,random_state=100,
                                           update_every=1,chunksize=10,
                                           passes=5,alpha='symmetric',
                                           iterations=100,per_word_topics=True)

lda_model.print_topics()[0][1].split(' + ')

/Users/lijianwei/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.

/Users/lijianwei/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.

/Users/lijianwei/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.

/Users/lijianwei/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. 

['0.180*"impathetic"',
 '0.180*"criminalmind"',
 '0.107*"catch"',
 '0.107*"criminal"',
 '0.107*"mind"',
 '0.107*"scream"',
 '0.107*"loudly"',
 '0.107*"roomie"']

In [13]:
text2 = 'Catching Criminal Minds and I screamed so loudly my roomie came running in 😂😂 #impathetic #criminalminds'

In [23]:
text = "@youyouentertain Kudos for the wonderful compliment &amp; ❤️, Carlos! This review of “Nyet” came from P. Mugridge. He s… https://t.co/6Qc79oPju7"

In [15]:
def extract_emojis(text):
    return ''.join(c for c in text if c in emoji.UNICODE_EMOJI)

extract_emojis(text2)

'😂😂'

In [5]:


def extract_Uniemojis(text):
    return list(set(c for c in text if c in emoji.UNICODE_EMOJI))

extract_Uniemojis(text2)

['😂']

In [8]:
# load this in Window
TWITTER_KEY = os.getenv('TWITTER_KEY')
TWITTER_SECRET = os.getenv('TWITTER_SECRET')
TWITTER_TOKEN = os.getenv('TWITTER_TOKEN')
TWITTER_TOKEN_SECRET = os.getenv('TWITTER_TOKEN_SECRET')

In [6]:
# load this in Mac using apikey
from apikey import TWITTER_KEY,TWITTER_SECRET,TWITTER_TOKEN,TWITTER_TOKEN_SECRET

In [7]:
auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_SECRET)
auth.set_access_token(TWITTER_TOKEN, TWITTER_TOKEN_SECRET)

api = tweepy.API(auth)

In [24]:
class MyStreamListener(tweepy.StreamListener):

    latestTweets = []
    totalTweets = 0
    nExactLocation = 0
    startTime = None
    
    def on_status(self, status):
        
        analysis = TextBlob(status.text)
        
        self.latestTweets.insert(0,(status.text, analysis.sentiment[0], status.coordinates))
        self.totalTweets += 1
        
        if (self.startTime is None):
            self.startTime = time()
        
        if (status.coordinates):
            self.nExactLocation += 1
        
        if len(self.latestTweets) > 10:
            self.latestTweets.pop()
        
        for tweet in self.latestTweets:
            print(tweet)
            
        print()
        elapsedTime = time() - self.startTime
        print('Tweets with lat/long: {} of {} ({:.2f}%); Frequency: {:.2f} tweets/s'.format(self.nExactLocation,
                                                                                              self.totalTweets,
                                                                                              self.nExactLocation/self.totalTweets*100,
                                                                                              self.totalTweets/elapsedTime))
        
        clear_output(wait=True)
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [91]:
text2

'Catching Criminal Minds and I screamed so loudly my roomie came running in 😂😂 #impathetic #criminalminds'

In [93]:
text2.split('#')[1:]

['impathetic ', 'criminalminds']

In [11]:
class MyStreamListener2(tweepy.StreamListener):

    latestTweets = []
    totalTweets = 0
    nExactLocation = 0
    startTime = None
    emojis=[]
    
    def extract_hashtags(text):
        return 
        
        
    def extract_emojis(text):
        return ''.join(c for c in text if c in emoji.UNICODE_EMOJI)
    
    def on_status(self, status):
        
        analysis = TextBlob(status.text)
        
        self.latestTweets.insert(0,(extract_emojis(status.text),status.text, analysis.sentiment[0], status.coordinates))
        self.emojis.insert(0,(extract_emojis(status.text),analysis.sentiment[0],status.coordinates))
        

        self.totalTweets += 1
        
        if (self.startTime is None):
            self.startTime = time()
        
        if (status.coordinates):
            self.nExactLocation += 1
        
        if len(self.latestTweets) > 10:
            self.latestTweets.pop()
        
        for tweet in self.latestTweets:
            print(tweet)
        for emoji in self.emojis:
            pass
            
            
        print()
        elapsedTime = time() - self.startTime
        print('Tweets with lat/long: {} of {} ({:.2f}%); Frequency: {:.2f} tweets/s'.format(self.nExactLocation,
                                                                                              self.totalTweets,
                                                                                              self.nExactLocation/self.totalTweets*100,
                                                                                              self.totalTweets/elapsedTime))

        
        clear_output(wait=True)
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [9]:
myStreamListener = MyStreamListener2()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [10]:
#New York City bounding box according to Twitter Docs: -74,40,-73,41
#https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters
data = myStream.filter(locations=[-74.05,40.54,-73.7,40.92])

KeyboardInterrupt: 

In [88]:
import plotly.plotly as py

from plotly.graph_objs import *

In [89]:
py.sign_in('jianweili0', '4hIbqMlvMyG6KPuZuUWF')
trace1 = {
  "marker": {
    "opacity": 0.7, 
    "size": 16
  }, 
  "mode": "markers+text", 
  "name": "Tweet!<br>Hover to view text", 
  "stream": {
    "maxpoints": 8, 
    "token": "7hj1ng08ob"
  }, 
  "textposition": "bottom", 
  "type": "scatter"
    
}

In [90]:
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": -0.06, 
      "y": -0.141, 
      "showarrow": False, 
      "text": "The happiness of each tweet is computed with Dodds et al's happiness metric: <a href=\"http://arxiv.org/abs/1101.5120\">http://arxiv.org/abs/1101.5120</a>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "hovermode": "closest", 
  "legend": {
    "bgcolor": "rgba(255, 255, 255, 0)", 
    "bordercolor": "rgba(255, 255, 255, 0)"
  }, 
  "showlegend": False, 
  "title": "Real-Time Happiness of SF's Twittersphere", 
  "xaxis": {
    "showgrid": False, 
    "showline": False, 
    "ticks": "", 
    "title": "Time", 
    "zeroline": False
  }, 
  "yaxis": {
    "showgrid": True, 
    "showline": False, 
    "ticks": "", 
    "title": "Happiness Ranking of Tweet (0 is unhappy, 10 is happy)", 
    "zeroline": False
  }
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig)

/Users/lijianwei/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




ValueError: 
    Invalid value of type 'builtins.str' received for the 'textposition' property of scatter
        Received value: 'bottom'

    The 'textposition' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['top left', 'top center', 'top right', 'middle left',
            'middle center', 'middle right', 'bottom left', 'bottom
            center', 'bottom right']
      - A tuple, list, or one-dimensional numpy array of the above